we are getting there! so we first focus on breaking in front, any "in front", for each template sequence we extract a subgraph of possible nodes, then the subgraph of pre_post nodes (i dont think it matters distinguishing them) and then we further trim the graphs by taking only the valid paths seen in the data + extraction of the raw data pointers. another thing we could do is preparing a unique html page with that stuff!

# TODO: 
- these todo could also be fixed when we deal with cutin etc
- review the old dictionary-template matching function from OLD_03_search_function_and_visualisations.ipynb
- add possibilities of "*" values
- figure out an elegant way to deal with int-dict-index and str-dict-index
- merge the rendering of point nodes with image nodes

# IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import networkx as nx
import re
from copy import deepcopy
import os

In [ ]:
from imports.general_settings import *
from imports.general_functions import *
from imports.NGram_KNrealRecursion import recursive_NGramKneserNey
from pyvis.network import Network, Node, Edge

# UTILITY FUNCTIONS

In [ ]:
def patch_network_filenames(filename_stem) : 
    # output_path
    # inputPath_dictionaryImg
    # baseFolder
    fromPath = inputPath_dictionaryImg.replace('\\', '\\\\')
    toPath = "..\\\\" + "..\\\\" + inputFolder_dictionary + "\\\\" + inputFolder_dictionaryImg + "\\"
    
    with open(filename_stem + ".html", "r") as f:
        content = f.readlines()
    
    newContent = []
    for x in content :
        y = x
        while fromPath in y :
            #print(x)
            y = y[:y.find(fromPath)] + toPath + y[y.find(fromPath) + len(fromPath) : ]
        newContent.append(y)

    #for x in newContent :
    #    if fromPath in x :
    #        print(x)
    with open(output_path + filename_stem + ".html", "w") as f :
        f.writelines("".join(newContent))

In [ ]:
def extract_breaking_in_front_nodes() :
    dict_valid_scenario_nodes = []
    print(len(dictionary))
    for i in range(len(dictionary)) :
        print(i, 6*' ', end='\r')
        entry = dict(dictionary.iloc[i])
        for t in range(8) :
            if entry['target_' + str(t) + '_relative_position_lateral'] == 'same_lane' :
                if entry['target_' + str(t) + '_relative_position_longitudinal'] in ['0<=x<=10', '10<x<=30'] :
                    if entry['target_' + str(t) + '_longitudinal'] == 'd' :
                        dict_valid_scenario_nodes.append(i)
                        break
    dict_valid_scenario_nodes = list(set(dict_valid_scenario_nodes))
    print()
    print(len(dict_valid_scenario_nodes))
    
    dict_valid_scenario_nodes = [str(x) for x in dict_valid_scenario_nodes]

    return dict_valid_scenario_nodes
    

# SETTINGS

In [ ]:
raw_input_filename = "20170524_PP_01_Run_1.hdf5"

In [ ]:
stem_filename = raw_input_filename[:raw_input_filename.find(".hdf5")]

In [ ]:
stem_filename

In [ ]:
data_folder = '..\\..\\data\\'
baseFolder = data_folder + projectName + '\\'

In [ ]:
inputFolder_dictionary = "01_activity_dictionary"

In [ ]:
dictionary_filename_stem = "activityDictionary"
dataset_filename_stem = "mappedDataset"
nGramGraph_filename_stem = "nGramGraph"
networkX_filename_stem = "networkx"

In [ ]:
dataset_subnetwork_filename_stem = "dataset_subnetwork"
dataset_subnetwork_img_filename_stem = "dataset_subnetwork_img"
full_dataset_network_filename_stem = "full_dataset_network"
scenario_subnetwork_img_filename_stem = "scenario_subnetwork_img"

In [ ]:
gridSizeSubfolder = str(len(valid_relative_positions['lateral'])-1) + "x" + str(len(valid_relative_positions['longitudinal'])-1)

In [ ]:
inputFolder_dictionaryImg = "img_" + gridSizeSubfolder + '\\'

In [ ]:
imgExtension = ".jpg"

In [ ]:
dictionary_filename = dictionary_filename_stem + "_" + gridSizeSubfolder + ".csv"

In [ ]:
inputPath_dictionary = baseFolder + inputFolder_dictionary + "\\"
inputPath_dictionaryImg = inputPath_dictionary + inputFolder_dictionaryImg

In [ ]:
inputPath_dictionaryImg

In [ ]:
output_folder = "03_retrieved_scenarios"

In [ ]:
output_path = baseFolder + output_folder + "\\" + stem_filename + "\\"

In [ ]:
if output_folder not in os.listdir(baseFolder) :
    print("creating", baseFolder + output_folder)
    os.mkdir(baseFolder + output_folder)
else :
    print("folder", output_folder, "already exists")

In [ ]:
if stem_filename not in os.listdir(baseFolder + output_folder) :
    print("creating", stem_filename)
    os.mkdir(output_path)
else :
    print("folder", stem_filename, "already exists")

# DICTIONARY LOADING

In [ ]:
dictionary = pd.read_pickle(inputPath_dictionary + dictionary_filename)

In [ ]:
dictionary.shape

In [ ]:
#idx = np.random.randint(len(dictionary))
#img = plt.imread(inputPath_dictionaryImg + str(idx) + imgExtension)
#img = np.zeros(shape=img.shape) + img
#render(img, title=idx)

In [ ]:
#dict_entry = dict(dictionary.iloc[idx])

# let's "fake" a brake in front search
simple: same lane, lonfitudinal > 0, breaking

In [ ]:
dict_valid_scenario_nodes = extract_breaking_in_front_nodes()

In [ ]:
#dict_valid_scenario_nodes = [str(x) for x in dict_valid_scenario_nodes]

In [ ]:
print("the dictionary has", len(dict_valid_scenario_nodes), "nodes satisfying breaking in front")

In [ ]:
#for idx in dict_valid_scenario_nodes[:5] :
#    img = plt.imread(inputPath_dictionaryImg + str(idx) + imgExtension)
#    img = np.zeros(shape=img.shape) + img
#    render(img, title=idx)

# LOADING RAW DATASET/NETWORKS

In [ ]:
input_filename = dataset_filename_stem + "_" + stem_filename + ".csv"

In [ ]:
inputFolder_data = "02_mapped_dataset"
inputPath_data = baseFolder + inputFolder_data + "\\" + gridSizeSubfolder + "\\"

In [ ]:
dataframe = pd.read_csv(inputPath_data + input_filename, dtype='str')

In [ ]:
dataframe.shape

In [ ]:
nGramGraph_filename = nGramGraph_filename_stem + "_" + stem_filename + ".p"

In [ ]:
ngram_model = pickle.load(open(inputPath_data + nGramGraph_filename, 'rb'))

In [ ]:
networkX_filename = networkX_filename_stem + "_" + stem_filename + ".p"

In [ ]:
model_nx = pickle.load(open(inputPath_data + networkX_filename, "rb"))

In [ ]:
model_nx.number_of_nodes(), model_nx.number_of_edges()

In [ ]:
dataframe.head()

# DATASET NODE IDENTIFICATION
- we need to partition the dataset network into 3 node types:
    - scenario nodes
    - pre_post scenario nodes
    - non relevant nodes
- scenario nodes are the nodes in the dataset which are also part of dict_valid_scenario_nodes
- pre_post are extracted accordingly - predecessors

- we also represent the information via network colouring

## 1. find valid scenario nodes: all nodes in dataset which are part of dict_valid_scenario_nodes

In [ ]:
df_nodes = list(dataframe['dictionary_index'].drop_duplicates())

In [ ]:
#len(df_nodes)

In [ ]:
df_scenario_nodes = sorted(list(set(df_nodes).intersection(dict_valid_scenario_nodes)))

In [ ]:
print("the dataset has", len(df_scenario_nodes), "scenario nodes")

## 2. from the dataset network extract the pre/post scenario nodes
i.e. all predecessors/successors of scenario nodes which are not scenario nodes

In [ ]:
pre_nodes = []
for node in df_scenario_nodes :
    pre_nodes += list(set(model_nx.predecessors(node)).difference(df_scenario_nodes))
pre_nodes = list(set(pre_nodes))
len(pre_nodes)

In [ ]:
post_nodes = []
for node in df_scenario_nodes :
    
    post_nodes += list(set(model_nx.successors(node)).difference(df_scenario_nodes))
post_nodes = list(set(post_nodes))
len(post_nodes)

In [ ]:
df_pre_post_nodes = pre_nodes + post_nodes
df_pre_post_nodes = list(set(df_pre_post_nodes))
print("the dataset has", len(df_pre_post_nodes), "pre-post scenario nodes")

In [ ]:
df_remaining_nodes = list(set(model_nx.nodes()).difference(df_scenario_nodes + df_pre_post_nodes))

In [ ]:
print("remaining nodes:", len(df_remaining_nodes))

In [ ]:
len(df_remaining_nodes) + len(df_pre_post_nodes) + len(df_scenario_nodes) == model_nx.number_of_nodes()

In [ ]:
#df_scenario_nodes[:3]

In [ ]:
#df_pre_post_nodes[:3]

In [ ]:
#df_remaining_nodes[:3]

# 3. draw the full network with the 3 node types (pyvis)

In [ ]:
full_dataset_network = Network(height=500, width=1200, directed=True, notebook=False)

for node in df_remaining_nodes :
    full_dataset_network.add_node(int(node), node, color='black')
    
for node in df_pre_post_nodes :
    full_dataset_network.add_node(int(node), node, color='blue')

for node in df_scenario_nodes :
    full_dataset_network.add_node(int(node), node, color='green')

for edge in list(model_nx.edges()) :
    full_dataset_network.add_edge(int(edge[0]), int(edge[1]))

In [ ]:
full_dataset_network.save_graph(full_dataset_network_filename_stem + ".html")

# 4. subgraph extraction i.e. remove df_remaining_nodes

In [ ]:
scenario_subgraph_nx = model_nx.subgraph(df_pre_post_nodes + df_scenario_nodes)

In [ ]:
dataset_subnetwork = Network(height=500, width=1200, directed=True, notebook=False)

for node in df_pre_post_nodes :
    dataset_subnetwork.add_node(int(node), node, color='blue')

for node in df_scenario_nodes :
    dataset_subnetwork.add_node(int(node), node, color='green')

for edge in list(model_nx.edges()) :
    if edge[0] in df_pre_post_nodes and edge[1] in df_scenario_nodes :
        dataset_subnetwork.add_edge(int(edge[0]), int(edge[1]))
    elif edge[0] in df_scenario_nodes and edge[1] in df_pre_post_nodes :
        dataset_subnetwork.add_edge(int(edge[0]), int(edge[1]))
    elif edge[0] in df_scenario_nodes and edge[1] in df_scenario_nodes :
        dataset_subnetwork.add_edge(int(edge[0]), int(edge[1]))

for i in range(len(dataset_subnetwork.edges)) :
    dataset_subnetwork.edges[i]['color'] = edgeProperties['color']

In [ ]:
dataset_subnetwork.save_graph(dataset_subnetwork_filename_stem + ".html")

# 5. subgraph rendering with images

In [ ]:
dataset_subnetwork_img = deepcopy(dataset_subnetwork)

In [ ]:
for i in range(len(dataset_subnetwork_img.nodes)) :
    node_id = dataset_subnetwork_img.nodes[i]['id']
    dataset_subnetwork_img.nodes[i]['shape'] = 'image'
    dataset_subnetwork_img.nodes[i]['image'] = inputPath_dictionaryImg + str(node_id) + imgExtension
    
for i in range(len(dataset_subnetwork_img.edges)) :
    dataset_subnetwork_img.edges[i]['color'] = edgeProperties['color']

In [ ]:
dataset_subnetwork_img.save_graph(dataset_subnetwork_img_filename_stem + ".html")

# SCENARIO INSTANCE EXTRACTION
it's a multilevel thingie:
- the dataset now should ahve labels as "pre_post", "scenario", "nothing"
- then we can shrink the dataset to have unique repetitions for the sequence
- in that way we can just extract sequencese "pre_post"-"scenario"-"pre-post"
- we will have to see who we can deal with multi sequence scenarios like cut-in, as a node can be in both sequences (imagine 2 targets performing cut-in one after the other

In [ ]:
scenario_indices = ['n', 'p', 's']
scenario_indices_long = ['nothing', 'pre_post', 'scenario']

In [ ]:
def assign_scenario_label(dict_index) :
    x = str(dict_index)
    if x in df_scenario_nodes :
        return scenario_indices.index("s")
    elif x in df_pre_post_nodes :
        return scenario_indices.index("p")
    else :
        return scenario_indices.index("n")

In [ ]:
dataframe['scenario_label'] = dataframe['dictionary_index'].apply(lambda x : assign_scenario_label(x))

In [ ]:
tmpDf = dataframe.loc[dataframe['scenario_label'].diff() != 0]
no_dup_dataframe = pd.DataFrame()
no_dup_dataframe['df_idx'] = list(tmpDf.index)
no_dup_dataframe['scenario_idx'] = list(tmpDf['scenario_label'])
no_dup_dataframe['scenario_str'] = [scenario_indices[x] for x in  list(no_dup_dataframe['scenario_idx'])]
no_dup_dataframe['dic_idx'] = list(tmpDf['dictionary_index'])

In [ ]:
len(no_dup_dataframe)

# mining!

In [ ]:
scenario_sequence = ''.join(no_dup_dataframe['scenario_str'])

In [ ]:
scenario_sequence[:5]

In [ ]:
scenario_instances = [m.start() for m in re.finditer('psp', scenario_sequence)]

In [ ]:
print("i found", len(scenario_instances), "instances of breaking in front!")

In [ ]:
#fig = plt.figure()
#plt.plot(no_dup_dataframe['scenario_idx'])
#plt.scatter(x=scenario_instances, y=[2 for _ in scenario_instances], c='r')
#plt.yticks(np.arange(len(scenario_indices)), scenario_indices_long)
#plt.show()

# scenario-to-raw-indices retrieval!

In [ ]:
dataframe['dictionary_index_int'] = dataframe['dictionary_index'].apply(lambda x : int(x))

In [ ]:
scenario_retrieval_df = pd.DataFrame(columns=['init_pre_idx', 'init_scenario_idx', 'init_post_idx', 'end_post_idx',
                                              'init_pre_ts', 'init_scenario_ts', 'init_post_ts', 'end_post_ts',
                                              'pre_scenario_post_dict_sequence', 'scenario_dict_sequence'])
for si in scenario_instances :
    mini_df = no_dup_dataframe.loc[si:si+len('sps')]
    init_pre_idx = mini_df.iloc[0]['df_idx']
    init_scenario_idx = mini_df.iloc[1]['df_idx']
    init_post_idx = mini_df.iloc[2]['df_idx']
    end_post_idx = mini_df.iloc[3]['df_idx']
    
    init_pre_ts = float(dataframe.loc[init_pre_idx, 'timestamp'])
    init_scenario_ts = float(dataframe.loc[init_scenario_idx, 'timestamp'])
    init_post_ts = float(dataframe.loc[init_post_idx, 'timestamp'])
    end_post_ts = float(dataframe.loc[end_post_idx, 'timestamp'])
    
    tmp_df = dataframe.loc[init_pre_idx:end_post_idx-1]
    dict_seq = list(tmp_df.loc[tmp_df['dictionary_index_int'].diff() != 0]['dictionary_index'])
    dict_seq_full = '-'.join(dict_seq)
    dict_seq_str = '-'.join(dict_seq[1:-1])
    
    scenario_retrieval_df = scenario_retrieval_df.append({'init_pre_idx' : init_pre_idx, 
                                                          'init_scenario_idx' : init_scenario_idx, 
                                                          'init_post_idx' : init_post_idx, 
                                                          'end_post_idx' : end_post_idx,
                                                          'init_pre_ts' : init_pre_ts, 
                                                          'init_scenario_ts' : init_scenario_ts, 
                                                          'init_post_ts' : init_post_ts, 
                                                          'end_post_ts' : end_post_ts,
                                                          'pre_scenario_post_dict_sequence' : dict_seq_full,
                                                          'scenario_dict_sequence' : dict_seq_str
                                                         }, ignore_index=True)
    #break

In [ ]:
scenario_retrieval_df.to_csv(output_path + "scenario_instances.csv", index=False)

# scenario instance visualisation as graph (nah for now?)

In [ ]:
def get_scenario_subnetwork_img_node(lbl) :
    node = None
    for n in dataset_subnetwork_img.nodes :
        if n['label'] == lbl :
            node = n
            break
    return node

In [ ]:
scenario_subnetwork_img = Network(height=500, width=1200, directed=True, notebook=False)

added_nodes = []
for i in range(len(scenario_retrieval_df)) :
    print(i, '/', len(scenario_retrieval_df), end='\r')
    
    seq = scenario_retrieval_df.iloc[i]['pre_scenario_post_dict_sequence']
    seq = seq.split('-')
    
    # add nodes
    for j in range(len(seq)) :
        
        node_label = seq[j]
        if node_label not in added_nodes :
            # new node to add
            added_nodes.append(node_label)
            # get the style and augment
            node = get_scenario_subnetwork_img_node(node_label)
            colour = 'green'
            if j == 0 or j == len(seq) - 1 :
                colour = 'blue'
        
            node['color'] = {'border' : colour, 'highlight' : colour}
            node['borderWidth'] = 10
            node['borderWidth'] = 4
            node['borderWidthSelected'] = 4
            node['shapeProperties'] = {'useBorderWithImage' : True}
            
            scenario_subnetwork_img.add_node(int(node_label))
            
            for x in scenario_subnetwork_img.nodes :
                if x['id'] == int(node_label) :
                    for k in node.keys() :
                        x[k] = node[k]
            
        #break
    
    # add edges
    for j in range(len(seq) - 1) :
        from_node = int(seq[j])
        to_node = int(seq[j+1])
        found = False
        for e in scenario_subnetwork_img.edges :
            if e['from'] == from_node and e['to'] == to_node :
                found = True
                break
           
        if not found :
            # we can add an edge
            scenario_subnetwork_img.add_edge(from_node, to_node)
            for i in range(len(scenario_subnetwork_img.edges)) :
                if scenario_subnetwork_img.edges[i]['from'] == from_node and scenario_subnetwork_img.edges[i]['to'] == to_node :
                    # fix style
                    scenario_subnetwork_img.edges[i]['color'] = edgeProperties['color']
            
        #break
    
        
    #break
print("\n.")

In [ ]:
scenario_subnetwork_img.save_graph(scenario_subnetwork_img_filename_stem + ".html")

# patching the graphs to then move them to the appropriate folder

In [ ]:
for f in [dataset_subnetwork_filename_stem, dataset_subnetwork_img_filename_stem, full_dataset_network_filename_stem, scenario_subnetwork_img_filename_stem] :
    patch_network_filenames(f)

In [ ]:
print("done!")